In [1]:
import sys

import matplotlib.pyplot as plt
import numpy as np

sys.path.append("../")
from metrics import f_score, auc_roc
from models import LinearSVM
from visualizations import plot_roc, plot_losses

%load_ext autoreload
%autoreload 2

In [2]:
train = np.load("../data/dataset_prep/train.npz")
x_train = train["x_train"]
y_train = train["y_train"]

In [3]:
x_train.shape, y_train.shape
# shape is (N, D)

((328135, 622), (328135,))

In [4]:
from preprocessing import COL_REMOVE_MANUAL, impute_missing_values, remove_duplicate_columns

# copied and altered `normalize_and_bias_data` from preprocessing.py to make it suitable for neural networks
# alterations
# > no need for bias since neuralnet already does it
# > no need for squared features since neural net can learn non-linear relationships

variable_type = []
with open("../data/metadata/variable_type.txt", "r") as f:
    for line in f:
        line = line.strip().strip('"')  # remove whitespace and surrounding quotes
        # split by comma and convert to int
        if line == "":
            variable_type.append([])
        else:
            variable_type.append(line)

# def normalize_and_bias_data(x_train, x_test=None, squared_features=True, feature_names=None):
def normalise_data(x_train, x_test=None, feature_names=None):

    """Standardize data and add bias term.
    Args:
        x_train: np.ndarray of shape (N_train, D)
        x_test: np.ndarray of shape (N_test, D)
    Returns:
        x_train_std: np.ndarray of shape (N_train, D+1)
        x_test_std: np.ndarray of shape (N_test, D+1)
    """ 
    if feature_names is None: feature_names = np.arange(x_train.shape[1])
    one_hot_encoding = x_train.shape[1] > len(variable_type) # one-hot encoding was applied

    # missing data imputation
    x_train = impute_missing_values(x_train, x_train)
    if x_test is not None: x_test = impute_missing_values(x_train, x_test)
    
    # manual duplicates removal
    x_train = np.delete(x_train, COL_REMOVE_MANUAL, axis=1)
    remaining_var_types = np.delete(variable_type, COL_REMOVE_MANUAL)
    feature_names = np.delete(feature_names, COL_REMOVE_MANUAL)
    assert feature_names.shape[0] == x_train.shape[1], "Feature names and data shape mismatch after manual duplicate removal."

    # invariant feature removal
    stds_train = np.std(x_train, axis=0)
    remove = (stds_train < 1e-10)
    x_train = x_train[:, ~remove]
    remaining_var_types = np.delete(remaining_var_types, remove[:len(remaining_var_types)])
    feature_names = np.delete(feature_names, remove)
    assert feature_names.shape[0] == x_train.shape[1], "Feature names and data shape mismatch after invariant feature removal."

    # feature scaling by standardization
    mean = x_train.mean(axis=0)
    std = x_train.std(axis=0)
    assert not np.any(std == 0), "At least one feature has zero standard deviation."
    x_train = (x_train - mean) / std
    
    # # squaring continuous features
    # if squared_features:
    #     squared_feature_idx = np.array([i for i, var_type in enumerate(remaining_var_types) if var_type in ["ordinal", "continuous"]])
    #     x_train = np.column_stack([x_train, x_train[:, squared_feature_idx] ** 2])
    #     squared_feature_names = [f"{feature_names[i]}_squared" for i in squared_feature_idx]
    #     feature_names = np.concatenate([feature_names, squared_feature_names])
    # assert feature_names.shape[0] == x_train.shape[1], "Feature names and data shape mismatch after squaring features."
    
    if one_hot_encoding: 
        one_hot_encoded = [i for i, t in enumerate(remaining_var_types) if t == "nominal"]
        x_train = np.delete(x_train, one_hot_encoded, axis=1)
        feature_names = np.delete(feature_names, one_hot_encoded)
        assert feature_names.shape[0] == x_train.shape[1], "Feature names and data shape mismatch after removing one-hot encoded features."

    # # add bias term
    # x_train = np.c_[np.ones((x_train.shape[0], 1)), x_train]
    # feature_names = np.concatenate([np.array(["bias"]), feature_names])

    if x_test is None:
        x_train, kept_idx = remove_duplicate_columns(x_train)
        return x_train, feature_names[kept_idx]

    x_test = np.delete(x_test, COL_REMOVE_MANUAL, axis=1)
    x_test = x_test[:, ~remove]
    x_test = (x_test - mean) / std
    # if squared_features:
    #     x_test = np.column_stack([x_test, x_test[:, squared_feature_idx] ** 2])
    if one_hot_encoding: x_test = np.delete(x_test, one_hot_encoded, axis=1)
    # x_test = np.c_[np.ones((x_test.shape[0], 1)), x_test]
    x_train, x_test, kept_idx = remove_duplicate_columns(x_train, x_test)
    return x_train, x_test, kept_idx, feature_names[kept_idx]

In [8]:
from preprocessing import lazy_load_data

x_test_orig = None
lazy_load_data()
x_test = x_test_orig.copy()

x_train_norm, x_submission_test, test_ids, _ = normalise_data(x_train, x_test)
N = x_train_norm.shape[0]
D = x_train_norm.shape[1]

Loading raw data...


AttributeError: 'NoneType' object has no attribute 'copy'

In [ ]:
from numpy import floor

train_test_ratio = 0.9
split_index = int(N * train_test_ratio)

permutation = np.random.permutation(N) # randomising data incase of skew

x_train_reshape = x_train_norm.transpose()[:, permutation]
y_train_reshape = y_train.reshape(1, -1)[:, permutation]

x_train_final = x_train_reshape[:, :split_index]
y_train_final = y_train_reshape[:, :split_index]

x_test_local = x_train_reshape[:, split_index:N]
y_test_local = y_train_reshape[:, split_index:N]

x_train_final.shape, y_train_final.shape, x_test_local.shape, y_test_local.shape, y_train_final.shape, ratio

In [ ]:
# eww why does python not have inline funcs
def ReLU(Z):
    return np.maximum(0, Z)

In [ ]:
# i don't import binary_cross_entropy_loss from models, since it is not suitable for neuralnet
def binary_cross_entropy_loss(Y, A):
    # Y: (1, N)
    # A: (1, N)
    m = Y.shape[1]
    epsilon = 1e-9
    cost = -(1/m) * np.sum( Y * np.log(A + epsilon) + (1 - Y) * np.log(1 - A + epsilon) )
    return np.squeeze(cost)

In [ ]:
from implementations import sigmoid

class NeuralNet:
    
    # Output size will be 1 given it is a binary classification
    def __init__(self):
        self.layer_dims = None
        self.L = None
        self.learning_rate = None
        self.batch_size = None
        self.params = {}
        self.patience = None
        self.hyperparameter_log = []
        
    # note, X assumes shape (D, N) to perform multiple at the same time, returns logits of shape (1, N)
    def forward_propagation(self, X):
        cache = [] # need to store the history of all activations for backprop

        A = X # since the first layer is already normalised
        Z = X # just tmp

        for i in range(1, self.L):
            A_prev = A
            Z = np.dot(self.params[f"W{i}"], A_prev) + self.params[f"b{i}"]
            cache.append((A_prev, self.params[f"W{i}"], Z))
            if (i != self.L - 1): A = ReLU(Z)

        return Z, cache
    
    def predict(self, X):
        # X: (D, N)
        # output: (1, N)

        Z, _ = self.forward_propagation(X)
        return (Z > 0).astype(int)
    
    def reinitialise_params(self):
        self.params = {}
        for i in range(1, self.L): # start from 1, since 0 is input layer
                # Each column in weights matrix represents weights for one neuron, and small randomisation is necessary to break symmetry
                self.params[f"W{i}"] = np.random.randn(self.layer_dims[i], self.layer_dims[i-1]) * 0.01 # shape (curr, prev)
                self.params[f"b{i}"] = np.zeros((self.layer_dims[i], 1)) # shape (curr, 1)

    def hyperparameter_tuning(self, X, y, metric, verbose=False):
        D = X.shape[0]

        # In layer_dims, we have [input_size, hidden_sizes ..., output_size]
        self.layer_dims = [D, 16, 8, 1]
        self.L = len(self.layer_dims)
        self.learning_rate = 0.01

        layers = [[D, 128, 1], [D, 16, 8, 1], [D, 32, 16, 1]]
        batch_sizes = [16, 32]
        learning_rates = [0.01, 0.001]
        patiences = [3, 5, 10]

        for layer in layers:
            self.layer_dims = layer
            self.L = len(self.layer_dims)
            self.reinitialise_params()

            for batch_size in batch_sizes:
                self.batch_size = batch_size

                for learning_rate in learning_rates:
                    self.learning_rate = learning_rate

                    for patience in patiences:
                        self.patience = patience

                        print(f"layers: {self.layer_dims}, batch size: {self.batch_size}, learning rate: {self.learning_rate}, patience: {self.patience}")
                        costs, test_costs, _ = self.train(X, y)

                        y_pred = self.predict(X)
                        f1 = f_score(y, y_pred)

                        self.hyperparameter_log.append({
                            # "final_test_cost": test_costs[-1], # accidentally optimised for cost
                            "f1": f1,
                            "layers": self.layer_dims,
                            "batch_size": self.batch_size,
                            "learning_rate": self.learning_rate,
                            "patience": self.patience,
                        })

                        self.reinitialise_params()

        self.hyperparameter_log.sort(key=lambda x: x["f1"])
        best_hyperparams = self.hyperparameter_log[-1] 

        self.layer_dims = best_hyperparams["layers"]
        self.L = len(self.layer_dims)
        self.reinitialise_params()
        self.learning_rate = best_hyperparams["learning_rate"]
        self.batch_size = best_hyperparams["batch_size"]
        self.patience = best_hyperparams["patience"]

    def train(self, X, Y):
        # X: (D, N)
        # Y: (1, N)
        N = X.shape[1]
        batch_size = self.batch_size

        # oversampling the minority class due to the bias in the dataset
        ratio = int(floor(np.sum(Y == 0) / np.sum(Y == 1)))
        mask_0 = (Y[0] == 0)
        mask_1 = (Y[0] == 1)
        x_zeros = X[:, mask_0]
        x_ones = X[:, mask_1]
        y_zeros = Y[:, mask_0]
        y_ones = Y[:, mask_1]
        x_ones_oversampled = np.tile(x_ones, (1, ratio))
        y_ones_oversampled = np.tile(y_ones, (1, ratio))
        X = np.concatenate([x_zeros, x_ones_oversampled], axis=1)
        Y = np.concatenate([y_zeros, y_ones_oversampled], axis=1)

        # need to do another permutation so it is not just ordered as zeros then ones
        train_permutation = np.random.permutation(X.shape[1])
        X = X[:, train_permutation]
        Y = Y[:, train_permutation]

        cost_sample_rate_in_epochs = 1
        costs = []
        test_costs = []

        max_epochs= 100
        patience = self.patience # how many epochs to wait before giving up
        patience_counter = 0 

        for i in range(max_epochs): 
            # need to do some shuffling to stop memorisation and overfitting
            permutation = np.random.permutation(N)
            X_shuffled = X[:, permutation]
            Y_shuffled = Y[:, permutation]

            for j in range(int(np.ceil(N/batch_size))):
                start = j * batch_size
                end = min((j + 1) * batch_size, N)

                X_batch = X_shuffled[:, start:end]
                Y_batch = Y_shuffled[:, start:end]
                real_batch_size = end - start

                # 1. for each batch perform forward propagation (we don't need to find the cross entropy loss since it works with derivatives)
                Z, cache = self.forward_propagation(X_batch)
                A = sigmoid(Z) 
                # C = binary_cross_entropy_loss(Y_batch, A)

                dA_prev = None
                for l in range(self.L - 1, 0, -1):
                    # 2. backprop formulas condensed
                    A_prev, W, Z = cache[l-1]

                    is_last_layer = l == self.L - 1
                    dZ = A - Y_batch if is_last_layer else dA_prev * (Z > 0).astype(int) # this last term is the derivate (kind of) of ReLU
                    dW = (1 / real_batch_size) * np.dot(dZ, A_prev.T)
                    db = (1 / real_batch_size) * np.sum(dZ, axis=1, keepdims=True)
                    
                    dA_prev = np.dot(W.T, dZ)

                    # 3. Update weights and biases of the network
                    self.params[f"W{l}"] = self.params[f"W{l}"] - self.learning_rate * dW
                    self.params[f"b{l}"] = self.params[f"b{l}"] - self.learning_rate * db
                
            # Doing some printing and tracking off cost.
            if i % cost_sample_rate_in_epochs == 0: # i just decided to print every epoch ... so it doesn't really make sense to have in if-statement, but just in-case
                Z, _ = self.forward_propagation(X) 
                A = sigmoid(Z)
                cost = binary_cross_entropy_loss(Y, A)
                costs.append(cost)
                print(f"Epoch {i}, Cost: {cost}")

                Z, _ = self.forward_propagation(x_test_local)
                A = sigmoid(Z)
                test_cost = binary_cross_entropy_loss(y_test_local, A)
                test_costs.append(test_cost)
                print(f".          Test Cost: {test_cost}")

            if test_cost < min(test_costs):
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience: # give up if no improvement 
                    break

        return costs, test_costs, cost_sample_rate_in_epochs


In [ ]:
nn = NeuralNet()
nn.hyperparameter_tuning(x_train_final, y_train_final, metric="loss", verbose=True)
nn.train(x_train_final, y_train_final)
predictions = nn.predict(x_submission_test)


In [ ]:
import os

from helpers import create_csv_submission

# Convert to -1 and 1
predictions = np.where(predictions == 1, 1, -1)

# Create submission file
os.makedirs("data/submissions", exist_ok=True)
create_csv_submission(test_ids, predictions, f"data/submissions/neural_network.csv")

# ===

In [ ]:
D = x_train_final.shape[0]

nn = NeuralNet([D, 12, 8, 1], learning_rate=0.001)
costs, test_costs, cost_sample_rate_in_epochs = nn.train(x_train_final, y_train_final, epochs=20)

In [ ]:
epochs = np.arange(1, len(costs) + 1) * cost_sample_rate_in_epochs
plt.plot(epochs, costs, marker='o', linestyle='-', label='Train Cost')
plt.plot(epochs, test_costs, marker='x', linestyle='--', label='Test Cost')

plt.title('Training & Test Cost vs. Epochs')
plt.xlabel(f'Epoch (Sampled every {cost_sample_rate_in_epochs})')
plt.ylabel('Cost')
plt.legend()
plt.show()

In [ ]:
y_pred = nn.predict(x_test_local)
print("F1 Score:", f_score(y_test_local, y_pred))
y_pred.shape, y_test_local.shape